<a href="https://colab.research.google.com/github/Benjamin-Menashe/courses/blob/master/Copy_of_Please_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import numpy as np
import torch.optim as optim
import pandas as pd
import matplotlib as plt
from google.colab import drive

drive.mount('/content/drive')
np.random.seed(123)

Mounted at /content/drive


In [28]:
x = pd.read_csv("drive/MyDrive/Colab_Notebooks/NN_project/Clean_3.csv")
sub_list = x['Participant_ID'].unique()

In [30]:
dt = torch.zeros(55*84, 60, 32)
j = 0

for i in sub_list:
  for t in range(1,56):
    cur = x.query(f"Participant_ID == '{i}' & Text_ID=={t}").filter(items=['Word_Length','Syl','CV_Complexity','Content','Noun','Verb','Adjective','Preposition','Article','Adverb','Pronoun','Determiner','Conjunction','Certainty','LSA_Context_Score','IA_AREA_Z','Freq','Orth','Orth_F','N1_F','N1_C','N2_F','N2_C','N3_F','N3_C','UN1_F','UN1_C','UN2_F','UN2_C','UN3_F','UN3_C','IA_FIXATION'])
    # cur = x.query(f"Participant_ID == '{i}' & Text_ID=={t}").filter(items=['Word_Length','Content','LSA_Context_Score','IA_FIXATION'])
    dt[j,:len(cur),:] = torch.tensor(cur.values).reshape(1,len(cur),32)
    j += 1


In [31]:
x = dt[:,:,0:31]
y = dt[:,:,31].reshape(4620,60,1)

In [45]:
x = x.nan_to_num()
y = y.nan_to_num()

rnning

In [36]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset

In [60]:
train = TensorDataset(x[0:3700,:,:],y[0:3700,:,:])
test = TensorDataset(x[3700:,:,:],y[3700:,:,:])

In [58]:
class VanillaRNN(nn.Module):
    def __init__(self, batch_size, input_size, hidden_size, output_size):
        super(VanillaRNN, self).__init__()
        self.batch_size, self.input_size, self.hidden_size, self.output_size = batch_size, input_size, hidden_size, output_size
        
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_size, num_layers=5, nonlinearity='relu')
        # Fully Connected Layer
        self.layer = nn.Linear(hidden_size, self.output_size)
        # Sigmoid
        self.layer2 = nn.Sigmoid()
    
    def forward(self, texts, prints=False):
        if prints: print('Original Texts Shape:', texts.shape)
        
        # Initialize hidden state with zeros
        hidden_state = torch.zeros(5, self.batch_size, self.hidden_size)
        if prints: print('Initial hidden state Shape:', hidden_state.shape)
        
        # Creating RNN
        hidden_outputs, hidden_state = self.rnn(texts, hidden_state)
        
        # 
        mid = self.layer(hidden_outputs.detach())
        out = self.layer2(mid)
        
        if prints:
            print('----hidden_outputs shape:', hidden_outputs.shape, '\n' +
                  '----final hidden state:', hidden_state.shape, '\n' +
                  '----out shape:', out.shape)
        
        if prints: print('Out Final Shape:', out.shape)
        
        return out

In [39]:
train_loader = DataLoader(train, batch_size=64)

In [53]:
# ==== STATICS ====
batch_size = 64        # how many images to be trained in one iteration
input_size = 31        # 
hidden_size = 100      # can be changed to any number: neurons
output_size = 1       # 
# =================

# Select one full batch from the data
texts_example, labels_example = next(iter(train_loader))
texts_example = texts_example.transpose(0,1)
print('original texts shape:', texts_example.shape)

model_example = VanillaRNN(batch_size, input_size, hidden_size, output_size)

out = model_example(texts_example, prints=True)

original texts shape: torch.Size([60, 64, 31])
Original Texts Shape: torch.Size([60, 64, 31])
Initial hidden state Shape: torch.Size([4, 64, 100])
----hidden_outputs shape: torch.Size([60, 64, 100]) 
----final hidden state: torch.Size([4, 64, 100]) 
----out shape: torch.Size([60, 64, 1])
Out Final Shape: torch.Size([60, 64, 1])


In [42]:
def get_accuracy(out, actual_labels, batchSize):
    predictions = out.round()
    correct = (predictions == actual_labels).sum().item()
    accuracy = correct/batch_size
    
    return accuracy

In [55]:
def train_network(model, train_data, test_data, batchSize=64, num_epochs=1, learning_rate=0.01):
    
    '''Trains the model and computes the average accuracy for train and test data.'''
    
    print('Get data ready...')
    # Create dataloader for training dataset - so we can train on multiple batches
    # Shuffle after every epoch
    train_loader = DataLoader(dataset=train_data, batch_size=batchSize, shuffle=True, drop_last=True)
    test_loader = DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, drop_last=True)
    
    # Create criterion and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)    
    
    print('Training started...')
    # Train the data multiple times
    for epoch in range(num_epochs):
        
        # Save Train and Test Loss
        train_loss = 0
        train_acc = 0
        
        # Set model in training mode:
        model.train()
        
        for k, (texts, labels) in enumerate(train_loader):
            
            # reshape
            texts = texts.transpose(0,1)
            labels = labels.transpose(0,1)

            # Create outputs
            out = model(texts)
            # Clears the gradients from previous iteration
            optimizer.zero_grad()
            # Computes loss: how far is the prediction from the actual?
            loss = criterion(out, labels)
            # Computes gradients for neurons
            loss.backward()
            # Updates the weights
            optimizer.step()
            
            # Save Loss & Accuracy after each iteration
            train_loss += loss.item()
            train_acc += get_accuracy(out, labels, batchSize)
            
        
        # Print Average Train Loss & Accuracy after each epoch
        print('TRAIN | Epoch: {}/{} | Loss: {:.2f} | Accuracy: {:.2f}'.format(epoch+1, num_epochs, train_loss/k, train_acc/k))
            
            
    print('Testing Started...')
    # Save Test Accuracy
    test_acc = 0
    # Evaluation mode
    model.eval()
    
    for k, (texts, labels) in enumerate(test_loader):
        
        # reshape
        texts = texts.transpose(0,1)
        labels = labels.transpose(0,1)

        # Create = predictions
        out = model(texts)
        # Add Accuracy of this batch
        test_acc += get_accuracy(out, labels, batchSize)
        
    # Print Final Test Accuracy
    print('TEST | Average Accuracy per {} Loaders: {:.5f}'.format(k, test_acc/k) )

In [61]:
# ==== STATICS ====
batch_size=64
input_size=31
hidden_size=150
output_size=1

# Instantiate the model
vanilla_rnn = VanillaRNN(batch_size, input_size, hidden_size, output_size)

# ==== TRAIN ====
train_network(vanilla_rnn, train, test, num_epochs=100)

Get data ready...
Training started...
TRAIN | Epoch: 1/100 | Loss: 0.28 | Accuracy: 44.50
TRAIN | Epoch: 2/100 | Loss: 0.27 | Accuracy: 44.91
TRAIN | Epoch: 3/100 | Loss: 0.27 | Accuracy: 44.91
TRAIN | Epoch: 4/100 | Loss: 0.27 | Accuracy: 44.92
TRAIN | Epoch: 5/100 | Loss: 0.28 | Accuracy: 44.55
TRAIN | Epoch: 6/100 | Loss: 0.27 | Accuracy: 44.82
TRAIN | Epoch: 7/100 | Loss: 0.27 | Accuracy: 44.93
TRAIN | Epoch: 8/100 | Loss: 0.27 | Accuracy: 44.91
TRAIN | Epoch: 9/100 | Loss: 0.27 | Accuracy: 44.91
TRAIN | Epoch: 10/100 | Loss: 0.27 | Accuracy: 44.89
TRAIN | Epoch: 11/100 | Loss: 0.27 | Accuracy: 44.92
TRAIN | Epoch: 12/100 | Loss: 0.27 | Accuracy: 44.91
TRAIN | Epoch: 13/100 | Loss: 0.27 | Accuracy: 44.91
TRAIN | Epoch: 14/100 | Loss: 0.27 | Accuracy: 44.91
TRAIN | Epoch: 15/100 | Loss: 0.27 | Accuracy: 44.91
TRAIN | Epoch: 16/100 | Loss: 0.27 | Accuracy: 44.91
TRAIN | Epoch: 17/100 | Loss: 0.27 | Accuracy: 44.90
TRAIN | Epoch: 18/100 | Loss: 0.27 | Accuracy: 44.92
TRAIN | Epoch: 19

KeyboardInterrupt: ignored

In [33]:
out

tensor([[[0.4876],
         [0.4876],
         [0.4876],
         ...,
         [0.4876],
         [0.4876],
         [0.4876]],

        [[0.4913],
         [0.4913],
         [0.4913],
         ...,
         [0.4913],
         [0.4913],
         [0.4913]],

        [[0.4918],
         [0.4918],
         [0.4918],
         ...,
         [0.4918],
         [0.4918],
         [0.4918]],

        ...,

        [[0.4923],
         [0.4923],
         [0.4923],
         ...,
         [0.4923],
         [0.4923],
         [0.4923]],

        [[0.4923],
         [0.4923],
         [0.4923],
         ...,
         [0.4923],
         [0.4923],
         [0.4923]],

        [[0.4923],
         [0.4923],
         [0.4923],
         ...,
         [0.4923],
         [0.4923],
         [0.4923]]], grad_fn=<SigmoidBackward0>)